In [1]:
from dask.distributed import Client

In [1]:
import pandas as pd

In [13]:
tabula.__version__

'2.6.0'

In [7]:
import tabula

In [2]:
# file = "../data/[file_name].xlsx"
# xl = pd.ExcelFile(file)
# sheets = xl.sheet_names  # see all sheet names
# sheets

In [9]:
tabula.io.read_pdf?

In [11]:
file = "../data/LF20_F40ADD_220.pdf"
data_dict = tabula.io.read_pdf(file, pages='all')
data_dict

Got stderr: Feb 02, 2023 2:09:47 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider loadDiskCache
Feb 02, 2023 2:09:47 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>
Feb 02, 2023 2:09:48 PM org.apache.pdfbox.pdmodel.font.FileSystemFontProvider <init>



[   LANDFIRE Fire Behavior Fuel Model 40 Attribute Data Dictionary  \
 0                                                 NaN               
 1                                           Attribute               
 2                                                 NaN               
 3                                               VALUE               
 4                                               -9999               
 5                                                  91               
 6                                                  92               
 7                                                  93               
 8                                                  98               
 9                                                  99               
 10                                                101               
 11                                                102               
 12                                                103               
 13                 

In [3]:
file = "../data/LF20_F40_220.csv"
raw_data = pd.read_csv(file)
raw_data.head()

,VALUE,FBFM40,R,G,B,RED,GREEN,BLUE
0,91,NB1,104,104,104,0.407843,0.407843,0.407843
1,92,NB2,225,225,225,0.882353,0.882353,0.882353
2,93,NB3,255,237,237,1.000000,0.929412,0.929412
3,98,NB8,0,14,214,0.000000,0.054902,0.839216
4,99,NB9,77,110,112,0.301961,0.431373,0.439216


In [16]:
raw_data.shape

(46, 8)

In [14]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46 entries, 0 to 45
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   VALUE   46 non-null     int64  
 1   FBFM40  46 non-null     object 
 2   R       46 non-null     int64  
 3   G       46 non-null     int64  
 4   B       46 non-null     int64  
 5   RED     46 non-null     float64
 6   GREEN   46 non-null     float64
 7   BLUE    46 non-null     float64
dtypes: float64(3), int64(4), object(1)
memory usage: 3.0+ KB


In [9]:
list(set(blah).difference(del_15))

['b', 'a']

In [4]:
# check the column order
import pickle
X_train, X_test, y_train, y_test = pickle.load(open("../data/train_test_data.pkl", "rb"))

for col in X_train.columns:
    print(col)

peak_i15_mmh
contributingarea_km2
prophm23
dnbr1000
kf
Fine fuel load
SAV
Packing ratio
Extinction moisture content
LNDS_RISKS
fire_interval
SedUn
SuscFrac


In [3]:
# load model
import torch
import torch.nn as nn
import torch.optim as optim

In [4]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, dropout_rate):
        super(Net, self).__init__()
        
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_rate) # dropout to prevent overfitting
        
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_rate) # dropout to prevent overfitting
        
#         self.fc3 = nn.Linear(hidden_size, hidden_size)
#         self.relu3 = nn.ReLU()
#         self.dropout3 = nn.Dropout(dropout_rate) # dropout to prevent overfitting

#         self.fc4 = nn.Linear(hidden_size, hidden_size)
#         self.relu4 = nn.ReLU()
#         self.dropout4 = nn.Dropout(dropout_rate) # dropout to prevent overfitting
        
        self.fc_out = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid() # output to probability rather than bool
        
        # Initilize weights with Glorot
        # need to decide on uniform vs normal distribution
        # UNIFORM
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        # nn.init.xavier_uniform_(self.fc3.weight)
        # nn.init.xavier_uniform_(self.fc4.weight)
        nn.init.xavier_uniform_(self.fc_out.weight)
        
        # NORMAL
        # nn.init.xavier_normal_(self.fc1.weight)
        # nn.init.xavier_normal_(self.fc2.weight)
        # nn.init.xavier_normal_(self.fc3.weight)
        # nn.init.xavier_normal_(self.fc4.weight)
        # nn.init.xavier_normal_(self.fc_out.weight)
        
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.dropout1(out)
        
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.dropout2(out)
        
#         out = self.fc3(out)
#         out = self.relu3(out)
#         out = self.dropout3(out)
        
#         out = self.fc4(out)
#         out = self.relu4(out)
#         out = self.dropout4(out)
        
        out = self.fc_out(out)
        out = self.sigmoid(out)      

        return out

In [7]:
model_params = {
    'dropout_rate': 0.2,
     'features': [
         'peak_i15_mmh',
          'SuscFrac',
          'prophm23',
          'Fine fuel load',
          'Extinction moisture content',
          'SAV',
          'SedUn',
          'dnbr1000',
          'fire_interval',
          'contributingarea_km2',
          'Packing ratio',
          'LNDS_RISKS',
          'kf'],
     'hidden_size': 100,
     'lr': 0.01
}

input_size = 13
hidden_size = model_params['hidden_size']
output_size = 1
dropout_rate = model_params['dropout_rate']
learning_rate = model_params['lr']

In [8]:
model = Net(input_size, hidden_size, output_size, dropout_rate)

In [10]:
model.load_state_dict(torch.load("../model/df_model_two_hidden.pth"))

<All keys matched successfully>

In [11]:
model.eval

<bound method Module.eval of Net(
  (fc1): Linear(in_features=13, out_features=100, bias=True)
  (relu1): ReLU()
  (dropout1): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=100, out_features=100, bias=True)
  (relu2): ReLU()
  (dropout2): Dropout(p=0.2, inplace=False)
  (fc_out): Linear(in_features=100, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)>

In [12]:
model.state_dict()

OrderedDict([('fc1.weight',
              tensor([[-2.6873e-02, -2.4580e-01, -2.2315e-01,  ..., -6.5710e-02,
                       -7.8055e-01,  4.2797e-01],
                      [ 4.2614e-01, -6.6906e-01, -4.1993e-04,  ..., -1.3552e-01,
                       -3.0696e-01, -5.1577e-01],
                      [ 3.7397e-01,  9.9725e-02, -1.3402e-01,  ...,  1.0296e-01,
                        4.9119e-01,  4.1338e-01],
                      ...,
                      [-5.2493e-01,  2.6015e-01, -5.0051e-03,  ..., -6.9454e-02,
                        5.4605e-02,  3.4259e-01],
                      [-8.7998e-01, -6.4932e-01,  3.3507e-01,  ..., -1.0559e-01,
                       -2.9691e-01, -1.0083e+00],
                      [-2.5192e-01, -2.5571e-01, -4.5685e-01,  ..., -2.1588e-01,
                       -3.0663e-01,  8.9361e-01]])),
             ('fc1.bias',
              tensor([-0.2365, -0.2941, -0.2232, -0.8006, -0.0457, -0.5987, -0.4514, -0.0681,
                      -0.5087, -0.82

# Prep Notebooks Rename

In [ ]:
# old file names and new file names
# replaced in notebooks but haven't tested yet

files = {
    # current: new
    # prep_01
    "staley16_debrisflow.parquet": "data_v01_add_site_ids.parquet",
    # prep_02
    "staley16_sites_catchment_fuelpars_v3.parquet": "data_v02_sites_catchment_fuelpars.parquet",
    "staley16_observations_catchment_fuelpars_v3.parquet": "data_v03_observations_catchment_fuelpars.parquet",
    # prep 03
    "staley16_observations_catchment_fuelpars_rocktype_v3.parquet": "data_v04_rocktype.parquet",
    # prep 04
    "staley16_observations_catchment_fuelpars_rocktype_randn_v3.parquet", "data_v05_rocktype_randn.parquet",
    # prep 06
    "staley16_observations_catchment_fuelpars_rocktype_randn_rockAge_v4.parquet", "data_v06_geological_age.parquet",
    # prep 07
    "staley16_observations_catchment_fuelpars_rocktype_randn_lndslide_v5.parquet", "data_v07_landslide.parquet",
    # prep 08
    "staley16_observations_catchment_fuelpars_rocktype_randn_lndslide_fireinterval_v6.parquet", "data_v08_fire_interval.parquet",
    # prep 09
    "": "",
    "": "",
    "": "",
    "", "",
    "", "",
    "", "",
    "", "",
    
    
}

In [1]:
import pandas as pd

In [5]:
file = "../data/ofr20161106_appx-1.xlsx"
xl = pd.ExcelFile(file)
sheets = xl.sheet_names
raw_data = pd.read_excel(file, sheet_name=sheets[1])
raw_data.head()

,Fire Name,Year,Fire_ID,Fire_SegID,Database,State,UTM_Zone,UTM_X,UTM_Y,Response,...,Peak_I15_mm/h,Peak_I30_mm/h,Peak_I60_mm/h,ContributingArea_km2,PropHM23,dNBR/1000,KF,Acc015_mm,Acc030_mm,Acc060_mm
0,Buckweed,2007,bck,bck_1035,Training,CA,11,368133.5165,3823231.989,0,...,3.2,2.0,2.0,0.378767,0.217933,0.297853,0.250000,0.8,1.0,2.0
1,Buckweed,2007,bck,bck_1090,Training,CA,11,367871.0165,3822984.489,0,...,3.2,2.0,2.0,0.689615,0.061249,0.224896,0.250000,0.8,1.0,2.0
2,Buckweed,2007,bck,bck_1570,Training,CA,11,367503.5165,3821741.989,0,...,3.2,2.0,2.0,2.757312,0.042968,0.065537,0.248541,0.8,1.0,2.0
3,Buckweed,2007,bck,bck_235,Training,CA,11,371108.5165,3824991.989,0,...,1.6,1.2,0.8,0.613415,0.092164,0.141711,0.250000,0.4,0.6,0.8
4,Buckweed,2007,bck,bck_363,Training,CA,11,370763.5165,3824576.989,0,...,1.6,1.2,0.8,0.538875,0.058353,0.210158,0.250000,0.4,0.6,0.8


In [6]:
raw_data.isna().sum()

Fire Name                 0
Year                      0
Fire_ID                   0
Fire_SegID                0
Database                  0
State                     0
UTM_Zone                  0
UTM_X                     0
UTM_Y                     0
Response                  0
StormDate                 0
GaugeDist_m               0
StormStart              160
StormEnd                160
StormDur_H                0
StormAccum_mm             0
StormAvgI_mm/h            0
Peak_I15_mm/h           230
Peak_I30_mm/h           214
Peak_I60_mm/h           256
ContributingArea_km2      0
PropHM23                  0
dNBR/1000                77
KF                        0
Acc015_mm               230
Acc030_mm               214
Acc060_mm               256
dtype: int64

In [26]:
# I feel like we could impute these
raw_data[raw_data['dNBR/1000'].isna()]['Response'].value_counts()

0    59
1    18
Name: Response, dtype: int64

In [12]:
raw_data[raw_data['Peak_I15_mm/h'].isna()]['Response'].value_counts()

0    230
Name: Response, dtype: int64

In [13]:
raw_data[raw_data['Peak_I30_mm/h'].isna()]['Response'].value_counts()

0    208
1      6
Name: Response, dtype: int64

In [16]:
raw_data[raw_data['Peak_I30_mm/h'].isna()].shape

(214, 27)

In [15]:
raw_data[raw_data['Peak_I30_mm/h'].isna()]['Peak_I15_mm/h'].isna().sum()

185

In [22]:
raw_data[(raw_data['Peak_I30_mm/h'].isna()) & (raw_data['Peak_I15_mm/h'].notna())]['Response'].value_counts()

0    23
1     6
Name: Response, dtype: int64

In [23]:
raw_data[(raw_data['Peak_I60_mm/h'].isna()) & (raw_data['Peak_I15_mm/h'].notna())]['Response'].value_counts()

0    100
1     34
Name: Response, dtype: int64